# Encircling the regions of the pharmacogenomic landscape that determine drug response

In this notebook is detailed the process followed to obtain the Hotnet inputs for the paper *Encircling the pharmacologic regions that determines drug response*.

*Required packages*
- pandas
- numpy
- scipy
- h5py
- networkx
- tqdm
- matplotlib
- sklearn

**Please, be aware that HotNet runs in python2 and some of the above packages must be also installed in python2**

In [45]:
import sys
sys.path.insert(0, '../src')
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
from scipy import stats
import scripts as src
from ml_model.ml import ensembol_gene_symbol,uniprot_gene_symbol
src.make_output_folders()
import os

## Reading drug sensitivity and gene expression data


In [2]:
gene_data = pd.read_csv('../data/MM_log2RPKM_expression_123_20200918.txt', sep='\t')
patient_drug_data = pd.read_csv('../data/DrugSensitivity.tsv', sep='\t')
expr_df, drug_df = src.prepare_drug_gene(gene_data, patient_drug_data)


from (123, 60619) genes to (123, 8802) genes 
102 drug with gene expression
102 drug by at least 35 sample
102 drug at least one drug reponase DSS less than 5 except nan


## Getting drug-gene correlations

In [ ]:
corr_df = src.get_correlations(expr_df,drug_df)

### Finding z-corr cutoff (0.025 prop at each tail)

In [ ]:

zscore_cutoff = np.mean(src.find_zscore_cutoff(corr_df))

In [ ]:
### Getting significant correlations for each drug

In [ ]:
up_genes,dw_genes = src.get_drug2gene_correlations(corr_df,zscore_cutoff)

In [ ]:
up_genes_symbol = {drug:list(ensembol_gene_symbol(genes).values()) for drug,genes in up_genes.items()}
dw_genes_symbol = {drug:list(ensembol_gene_symbol(genes).values()) for drug,genes in dw_genes.items()}

## Finding Frequently Correlated Genes (FCG)

In [ ]:
up_fcg = src.get_fcg(corr_df,up_genes)
dw_fcg = src.get_fcg(corr_df,dw_genes)

## Computing Reactome enrichments

###
This script can be slow. Please, consider using multiprocessing

### Running Reactome GSEA for each drug

In [ ]:
#Getting reactome
reactome_genesets =src.get_reactome_genesets()

#Getting reactome universe
react_univ = set([y for x in reactome_genesets.values() for y in x])

#Getting final universe: GDSC & reactome
the_universe = react_univ & src.map_and_return_universe(corr_df.columns,src.get_ensbl2uniAC())

#Getting reactome genesets in the universe
reactome_genesets = {r:reactome_genesets[r]&the_universe for r in reactome_genesets}

#Mapping ensemble to uniprot
mapped_corr_df = src.map_corr_df_to_uniprot(corr_df,the_universe)

#Iterating across drugs
for drug in tqdm(mapped_corr_df.index.values):
    output_path = '../results/reactome_gsea/%s.tsv'%drug
    matrix = list(zip(mapped_corr_df.columns,mapped_corr_df.loc[drug]))
    sorted_matrix = sorted([[str(x[0]),float(x[1])] for x in matrix], key=lambda x: x[1], reverse=True)

    #Running GSEA
    src.run_gsea(sorted_matrix,reactome_genesets,output_path)

## Getting HotNet input files

### 1.Removing FCG from the correlations

In [ ]:
up_genes_noFCG,dw_genes_noFCG = {},{}

for drug,genes in up_genes.items():
    genes = genes.difference(up_fcg)
    up_genes_noFCG[drug] = genes

for drug,genes in dw_genes.items():
    genes = genes.difference(dw_fcg)
    dw_genes_noFCG[drug] = genes


### 2.Getting significantly enriched pathways for each drug

In [ ]:
drug2pwy = src.get_drug2pwy()

###  3.Reading string network and reactome genesets


In [ ]:
string = set([])
with open('../data/string/interactions.tsv','r') as f:
    for l in f:
        string.update(l.rstrip().split('\t'))

reactome_genesets = src.get_reactome_genesets()


### 4.Writing HotNet inputs

In [ ]:
#--up
output_path = '../results/hotnet_input/PCM/'
src.write_hotnet_inputs(up_genes_noFCG,corr_df,drug2pwy,reactome_genesets,string,output_path)

#--dw
output_path = '../results/hotnet_input/NCM/'
src.write_hotnet_inputs(dw_genes_noFCG,corr_df,drug2pwy,reactome_genesets,string,output_path)


## Running HotNet

This script can be slow. Please, consider using multiprocessing

In [ ]:
drugs = drug_df.columns
for direction in ['PCM','NCM']:
    for drug in tqdm(drugs,desc='%s'%direction):
        output = '../results/hotnet/%s/%s/'%(direction,drug)
        src.run_iteratively_hotnet('../results/hotnet_input/%s/%s.tsv'%(direction,drug),output)

## Running Diamond

This script can be slow. Please, consider using multiprocessing

In [ ]:
drugs = drug_df.columns

for direction in ['PCM','NCM']:
    module_path = '../results/hotnet/%s/'%direction
    for drug in tqdm(drugs,desc='%s'%direction):
        output = '../results/diamond/%s/'%direction
        src.run_diamond(module_path,output,sample=drug)

## Getting final modules¶

In [ ]:
#Getting modules
pcm = src.retrieve_iter_hotnet('../results/hotnet/PCM')
ncm = src.retrieve_iter_hotnet('../results/hotnet/NCM')

#Getting alpha list
diamond_pcm = src.read_diamond_res('../results/diamond/PCM/')
diamond_ncm = src.read_diamond_res('../results/diamond/NCM/')

#Getting new modules
final_pcm = src.add_diamond_genes(pcm,diamond_pcm,up_genes)
final_ncm = src.add_diamond_genes(ncm,diamond_ncm,dw_genes)

#Writing final modules
outpath = '../results/final_modules/'

# --PCM
with open(outpath+'/PCM.gmt','w') as o:
    for drug in sorted(final_pcm):
        for ix,md in enumerate(final_pcm[drug]):
            if ix == 0:
                o.write('%s\tna\t'%drug+'\t'.join(md)+'\n')
            elif ix == 1:
                o.write('%s_md2\tmodule_2\t'%drug+'\t'.join(md)+'\n')

# --NCM
with open(outpath+'/NCM.gmt','w') as o:
    for drug in sorted(final_ncm):
        for ix,md in enumerate(final_ncm[drug]):
            if ix == 0:
                o.write('%s\tna\t'%drug+'\t'.join(md)+'\n')
            elif ix == 1:
                o.write('%s_md2\tmodule_2\t'%drug+'\t'.join(md)+'\n')

In [ ]:
data_saved = pickle.load(open('../results/result.out', 'rb'))

final_pcm_symbol = {}
for d,uniprots in data_saved['final_pcm'].items():
    symbol_list = []
    for u in uniprots:
        symbol_list.append(list(uniprot_gene_symbol(u).values()))
    final_pcm_symbol[d] = symbol_list
data_saved['final_pcm_symbol'] = final_pcm_symbol

final_ncm_symbol = {}
for d,uniprots in data_saved['final_ncm'].items():
    symbol_list = []
    for u in uniprots:
        symbol_list.append(list(uniprot_gene_symbol(u).values()))
    final_ncm_symbol[d] = symbol_list

data_saved['final_ncm_symbol'] = final_ncm_symbol

with(open('../results/result.out', 'wb')) as handle:
    pickle.dump(data_saved, handle)

In [ ]:
data_saved = {}
for k, v in globals():
    if k.startswith(set(['gene', 'drug','up','final', 'ncm','dw','diamond','zscore','pcm','the_universe'])):
        data_saved.update(v)

with(open('../results/result.out', 'wb')) as handle:
    pickle.dump(data_saved, handle)

